In [ ]:
!pip install torchreid

In [ ]:
import torchreid
import torch
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# os.listdir('/content/drive/MyDrive/Resume/RadiusAi/')

# !mkdir /content/drive/MyDrive/Resume/RadiusAi/data

# !unzip '/content/drive/MyDrive/Resume/RadiusAi/dataset.zip' -d '/content/drive/MyDrive/Resume/RadiusAi/data'

In [ ]:
device = torch.device('cuda')


In [ ]:
extractor = torchreid.utils.FeatureExtractor('osnet_x0_25', 
                                             '/content/drive/MyDrive/Resume/RadiusAi/osnet_x0_25_msmt17.pt',
                                             device = str(device))
print(extractor.model)

In [ ]:
@torch.no_grad()
def run_inference(img, i):
  extractor(img)
  x = features['teats'].cpu().numpy()
  x = x[:,:,:,i].squeeze()
  # x = x.mean(axis = 2)
  img_resize = cv2.resize(x, (0, 0), fx=4, fy=1)


  heatmap_normalized = cv2.normalize(img_resize, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

  heatmap_color = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)

  # Set the weight for blending the heatmap and image
  heatmap_weight = 0.6  # Adjust this value to control the intensity of the heatmap overlay

  # Overlay the heatmap on the image
  output = cv2.addWeighted(img, 1 - heatmap_weight, heatmap_color, heatmap_weight, 0)
  
  cv2_imshow(output)

In [ ]:
imgs = os.listdir('/content/drive/MyDrive/Resume/RadiusAi/data/dataset')


In [ ]:
img1 = cv2.imread(os.path.join('/content/drive/MyDrive/Resume/RadiusAi/data/dataset',imgs[2]))
run_inference(img1, 1)

In [ ]:
import torchvision.transforms as T
from torch.nn import functional as F


In [ ]:
# Set the weight for blending the heatmap and image
heatmap_weight = 0.4  # Adjust this value to control the intensity of the heatmap overlay

# Overlay the heatmap on the image
mix = cv2.addWeighted(img1, 1 - heatmap_weight, am, heatmap_weight, 0)

In [ ]:
cv2_imshow(mix)

In [ ]:
def pre_process(input):
  image_size = (256, 128)
  transforms = []
  transforms += [T.Resize(image_size)]
  transforms += [T.ToTensor()]
  transforms += [T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]


  preprocess = T.Compose(transforms)

  to_pil = T.ToPILImage()

  image = to_pil(input)
  image = preprocess(image)
  images = image.unsqueeze(0).to(device)

  return images



In [ ]:
def post_process_am(activation):
  outputs = (activation**2).sum(1)
  b, h, w = outputs.size()
  outputs = outputs.view(b, h * w)
  outputs = F.normalize(outputs, p=2, dim=1)
  outputs = outputs.view(b, h, w)

  am = outputs[0, ...].detach().numpy()
  am = cv2.resize(am, (64, 128))
  am = 255 * (am - np.min(am)) / (
      np.max(am) - np.min(am) + 1e-12
  )
  am = np.uint8(np.floor(am))
  am = cv2.applyColorMap(am, cv2.COLORMAP_JET)

  return am

In [ ]:
def generate_visualization(img, am, intensity):

  # Set the weight for blending the heatmap and image
  heatmap_weight = intensity  # Adjust this value to control the intensity of the heatmap overlay

  # Overlay the heatmap on the image
  mix = cv2.addWeighted(img, 1 - heatmap_weight, am, heatmap_weight, 0)

  return mix

In [ ]:
def main(i):
  img = cv2.imread(os.path.join('/content/drive/MyDrive/Resume/RadiusAi/data/dataset',imgs[i]))
  activation = extractor.model(pre_process(img), True)
  activation_map = post_process_am(activation)
  output_img = generate_visualization(img, am, 0.6)
  cv2_imshow(output_img)

  return  

In [ ]:
main(200)

In [ ]:
img = cv2.imread(os.path.join('/content/drive/MyDrive/Resume/RadiusAi/data/dataset',imgs[200]))
output = extractor(img)


In [ ]:
!rm /content/drive/MyDrive/Resume/RadiusAi/data/dataset/.DS_Store

In [ ]:
lst = []
outputs =np.ones((20,32,512), dtype=np.float32)
for i in range(1, 20*32+1):
  names = os.path.join('/content/drive/MyDrive/Resume/RadiusAi/data/dataset',imgs[i])
  img = cv2.imread(names)
  lst.append(img)

  if (i)%32 == 0:
    output = extractor(lst)
    outputs[(i//32)-1, :, :] = output.numpy()
    lst.clear()


In [ ]:
x = outputs.reshape(20*32, 512)

In [ ]:
from numpy.linalg import norm

def cosine(A, B):
  
  # compute cosine similarity
  cosine = np.dot(A,B)/(norm(A)*norm(B))
  return cosine

In [ ]:
outer = []
master = list(range(160))

while(len(master)>0):
  key = master.pop(0)
  inner = [key]

  for i in master:
    if cosine(x[key, :], x[i, :]) > 0.75:
      inner.append(i)
      master.remove(i)
  outer.append(inner)

In [ ]:
imgs[1:161][0]

In [ ]:
imgs[1:161][53]

In [ ]:
imgs[1:161][62]

In [ ]:
for inner in outer:
  # n = len(inner)

  # tiles_x = n//2

  if len(inner) > 4:
    
    for idx in inner:
      cv2_imshow(cv2.imread(os.path.join('/content/drive/MyDrive/Resume/RadiusAi/data/dataset',imgs[1:161][idx])))

    break


In [ ]:
torchreid.utils.reidtools.visualize_ranked_results()